In [17]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy as np


In [18]:
pd.set_option('display.max_columns', 5000)
#pd.set_option('display.max_rows', None)

In [104]:
def getformations(dataframe): 
    df=dataframe['action_name']
    dft=dataframe['team_name']
    list=[]
    team=[]
    for i in range(4):
        x=df.iloc[i]
        teams = dft.iloc[i]
        if ('2'<x[0]<'6'):
            list+=[x]
            team+=[teams]
            
    return list, team

In [19]:
#instatiate
template_path='template.csv'
df=pd.read_csv(template_path)
df

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,carry.progression,carry.endLocation.x,carry.endLocation.y,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.opponent,location


In [136]:
#set all unknown attributes
id=matchId=relatedEventId=teamId=opponentTeamId=playerId=passrecipientId=possessionId=possessioneventsNumbe = 0
possessioneventIndex=possessionteamId=groundDuelopponentId=groundDuelrelatedDuelId=infractionopponentId=shotgoalkeeperActionId=shotgoalkeeperId=aerialDuelopponentId=aerialDuelrelatedDuelId = 0
passheight = None


In [132]:
#load instat file
instat_path = "data/InStat_AXA_Super_2023-06-02_Switzerland. AXA Women's Super League_Servette Chenois_vs_Zurich_2582307_2023-06-02_Servette Chenois_vs_Zurich_2582307_rawdata - Kopie.xml"
instatdf=pd.read_xml(instat_path)
instatdf=instatdf.iloc[2:,:]
instatdf.drop(columns=['column','number','dl', 'id', 'uid', 'action_id', 'player_id', 'team_id', 'standart_id', 'ts', 'position_id', 'opponent_id', 'opponent_team_id','opponent_position_id', 'zone_id', 'zone_dest_id', 'possession_id', 'possession_team_id', 'possession_team_id', 'attack_status_id', 'attack_type_id', 'attack_team_id', 'body_id' ], inplace=True)
mask = (instatdf['second']==0.00) & (~instatdf['player_name'].isna()) & (instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask]
todrop = ['line-up', 'Substitute player', '1st half','2nd half', 'Players, that created offside trap']
mask = ~instatdf['action_name'].isin(todrop)
instatdf = instatdf[mask]
l,t=getformations(instatdf)

In [134]:
l,t

(['4-2-3-1', '4-4-2 classic'], ['Servette Chenois', 'Zurich'])

In [133]:
instatdf

,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,direction,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,gate_x005F_x,gate_y,body_name,shot_type,touches,shot_handling,assistant_id,assistant_name
38,4-2-3-1,None,Servette Chenois,1.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
50,Attacking pass accurate,Paula Serrano Castano,Servette Chenois,1.0,0.00,Indirect free kick,Attacking midfielder - Central,Nahla Elodie Nakkach,Servette Chenois,Defensive midfielder - Right central,2C,2C,52.5,34.1,39.9,34.1,12.6,Start,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
51,4-4-2 classic,None,Zurich,1.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
153,Ball receiving,Nahla Elodie Nakkach,Servette Chenois,1.0,1.41,Open play,Defensive midfielder - Right central,None,None,None,2C,None,39.9,34.1,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
154,Attacking pass inaccurate,Nahla Elodie Nakkach,Servette Chenois,1.0,2.51,Open play,Defensive midfielder - Right central,None,None,None,2C,4L,41.8,33.3,105.0,56.3,67.3,Ball possession,Servette Chenois,0.0,1.0,0.9,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,Attacking pass accurate,Nahla Elodie Nakkach,Servette Chenois,2.0,2927.92,Open play,Defender - Left central,Maeva Jocelyne Clemaron,Servette Chenois,Defensive midfielder - Right central,2L,2L,28.7,54.1,34.8,56.3,6.5,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2635,Attacking pass accurate,Maeva Jocelyne Clemaron,Servette Chenois,2.0,2929.40,Open play,Defensive midfielder - Right central,Nahla Elodie Nakkach,Servette Chenois,Defender - Left central,2L,2L,34.8,56.3,32.5,63.2,7.3,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2636,Challenge,Nahla Elodie Nakkach,Servette Chenois,2.0,2930.33,Open play,Defender - Left central,Alayah Pilgrim,Zurich,Midfielder - Right,2L,None,32.5,63.2,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2637,Ball out of the field,None,Servette Chenois,2.0,2932.14,Open play,None,None,None,None,2L,None,31.2,68.0,NaN,NaN,NaN,End,Servette Chenois,10.0,239.0,NaN,End,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None


In [137]:
mask = (instatdf['second']==0)&(instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask] 
#ready for wyscout transformation

In [170]:
def create_event(instat, ind, wyscout):
    new_row = {col: pd.NA for col in wyscout.columns}
    wdf = wyscout.append(new_row, ignore_index=True)
    idf = instat
    index_instat = ind+1
    matchend=False
    return idf, index_instat, wdf

In [172]:
matchend=True
index = 0   
while (matchend):
    instatdf, index, df = create_event(instatdf, index, df)
    if(instatdf['action_name'].iloc[index]=='Match end'):
        break
     
    
    





done


In [ ]:
def parseaction(instat, ind, wyscout):
    newind=ind
    starttime
    return newind

In [125]:
instatdf

,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,direction,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,gate_x005F_x,gate_y,body_name,shot_type,touches,shot_handling,assistant_id,assistant_name
50,Attacking pass accurate,Paula Serrano Castano,Servette Chenois,1.0,0.00,Indirect free kick,Attacking midfielder - Central,Nahla Elodie Nakkach,Servette Chenois,Defensive midfielder - Right central,2C,2C,52.5,34.1,39.9,34.1,12.6,Start,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
153,Ball receiving,Nahla Elodie Nakkach,Servette Chenois,1.0,1.41,Open play,Defensive midfielder - Right central,None,None,None,2C,None,39.9,34.1,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
154,Attacking pass inaccurate,Nahla Elodie Nakkach,Servette Chenois,1.0,2.51,Open play,Defensive midfielder - Right central,None,None,None,2C,4L,41.8,33.3,105.0,56.3,67.3,Ball possession,Servette Chenois,0.0,1.0,0.9,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
155,Lost balls,Nahla Elodie Nakkach,Servette Chenois,1.0,2.51,Open play,None,None,None,None,4L,None,105.0,56.3,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
156,Ball out of the field,None,Servette Chenois,1.0,8.49,Open play,None,None,None,None,4L,None,105.0,57.8,NaN,NaN,NaN,End,Servette Chenois,8.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,Attacking pass accurate,Nahla Elodie Nakkach,Servette Chenois,2.0,2927.92,Open play,Defender - Left central,Maeva Jocelyne Clemaron,Servette Chenois,Defensive midfielder - Right central,2L,2L,28.7,54.1,34.8,56.3,6.5,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2635,Attacking pass accurate,Maeva Jocelyne Clemaron,Servette Chenois,2.0,2929.40,Open play,Defensive midfielder - Right central,Nahla Elodie Nakkach,Servette Chenois,Defender - Left central,2L,2L,34.8,56.3,32.5,63.2,7.3,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2636,Challenge,Nahla Elodie Nakkach,Servette Chenois,2.0,2930.33,Open play,Defender - Left central,Alayah Pilgrim,Zurich,Midfielder - Right,2L,None,32.5,63.2,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2637,Ball out of the field,None,Servette Chenois,2.0,2932.14,Open play,None,None,None,None,2L,None,31.2,68.0,NaN,NaN,NaN,End,Servette Chenois,10.0,239.0,NaN,End,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None


['Switzerland U17', 'Finland U17']

In [ ]:
kickoff_team_1 =False